In [3]:
import pickle
import cv2
import numpy as np
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def get_landmark(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    result = face_mesh.process(img)
    
    if result.multi_face_landmarks:
        face_landmarks = result.multi_face_landmarks[0]
        landmarks = []
        for lm in face_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z])
        return np.array(landmarks).reshape(1, -1)  # Make it 2D for predict()
    
    return None

def main():
    vid = cv2.VideoCapture(0)
    labels = ["angry", "happy", "neutral", "sad", "surprise"]

    with open("model_face.pkl", "rb") as f:
        model = pickle.load(f)

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        landmark = get_landmark(frame)
        if landmark is not None:
            res = model.predict(landmark)
            text = labels[res[0]]
            cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


I0000 00:00:1750436085.411357   21892 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750436085.422337   31217 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
W0000 00:00:1750436085.457134   31213 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750436085.523874   31214 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
